In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import os
import shutil
import time
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
train_df = pd.read_csv('../data/train_logs.csv')
train_scores = pd.read_csv('../data/train_scores.csv')
test_logs = pd.read_csv('../data/test_logs.csv')

In [5]:
cleaned_data = train_df.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby('id').sum() 
mins_data = train_df.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby("id").min()
maxes_data = train_df.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby("id").max()

cleaned_data['min_down_time'] = mins_data['down_time']
cleaned_data['max_up_time'] = maxes_data['up_time']
cleaned_data['min_action_time'] = mins_data['action_time']
cleaned_data['max_action_time'] = maxes_data['action_time']
cleaned_data = cleaned_data.merge(train_scores, on='id')


cleaned_test = test_logs.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby('id').sum()
mins_test = test_logs.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby("id").min() 
maxes_test = test_logs.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby("id").max()

cleaned_test['min_down_time'] = mins_test['down_time']
cleaned_test['max_up_time'] = maxes_test['up_time']
cleaned_test['min_action_time'] = mins_test['action_time']
cleaned_test['max_action_time'] = maxes_test['action_time']
cleaned_test = cleaned_test.merge(train_scores, on='id')
cleaned_test.head()

C:\Users\Nate\AppData\Local\Temp\ipykernel_12504\371043944.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cleaned_data = train_df.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby('id').sum()
C:\Users\Nate\AppData\Local\Temp\ipykernel_12504\371043944.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cleaned_test = test_logs.drop(columns=['cursor_position','down_event', 'up_event','text_change']).groupby('id').sum()


,id,event_id,down_time,up_time,action_time,word_count,min_down_time,max_up_time,min_action_time,max_action_time,score


In [4]:
drop_variables = ['id']
passthrough_variables = ['score']
scale_variables = ['event_id', 'down_time', 'up_time', 'action_time', 'word_count',
                   'min_down_time', 'max_up_time', 'min_action_time', 'max_action_time']

preprocessor = make_column_transformer(
    ('drop', drop_variables),
    ("passthrough", passthrough_variables),
    (StandardScaler(), scale_variables)
)

transformed = preprocessor.fit_transform(cleaned_data)
test_transformed = preprocessor.fit_transform(cleaned_test)
column_names = passthrough_variables + scale_variables

train = pd.DataFrame(transformed, columns=column_names)

X_train = train.drop(columns='score')
y_train = train.loc['score']

X_train_t = torch.tensor(X_train.values, dtype=torch.float32)
y_train_t = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test_t = torch.tensor(train.values, dtype=torch.float32)



ValueError: Found array with 0 sample(s) (shape=(0, 9)) while a minimum of 1 is required by StandardScaler.

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        return x
    
input_size = X_train_t.shape[1]
hidden_size = 30
output_size = 1
learning_rate = 0.05
num_epochs =100

model = Net(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    predicted = model(X_test_t)
    print(predicted.head())   